<a href="https://colab.research.google.com/github/CarlosfcPinheiro/pibic-api-llm-integration/blob/main/pibic_teste_aplicacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste de requisições para LLM utilizando padrão REST síncrono
O objetivo principal é testar e avaliar o tempo de resposta de requisições para uma API que interaja com uma LLM (atualmente https://huggingface.co/facebook/bart-large-cnn).

## Código e estrutura
Para análise, são utilizados 3 prompts diferentes, que buscam explorar diferentes tamanhos e temáticas de texto. São feitos 30 requisições para a API, onde são armazenado os tempos de resposta de todas as requests para cada prompt, respectivamente.

Após as requisições serem feitas, os tempos de requisição são armazenados em um csv, que é lido posteriormente para compor os dados de análise, que também são escritos em um outro csv.

## Padrões e bibliotecas
O padrão a ser utilizado é o **REST síncrono**, dado que aplica os princípios e restrições para criação de  sistemas distribuídos e escaláveis através de ações bloqueantes.

As principais bibliotecas utilizadas são:
- time (funções de manipulação de tempo)
- requests (envio de requisições HTTP)
- csv (manipulação de arquivos csv)
- statistics (funções para cálculos estatísticos)
- numpy (funções para manipulação de arrays e matrizes)

## Métricas analisadas
As métricas utilizadas para análise do tempo de resposta da requisição para cada prompt foram as seguintes:
- Média de tempo (tempo médio de resposta das requisições)
- Máximo e Mínimo (tempo com valores máximos e mínimos)
- Desvio padrão (dispersão absoluta dos dados em relação à média, numericamente)
- Coeficiente de variação (dispersão relativa dos dados em relação à média, em  porcentagem)
- Percentil - 95% e 99% (valores abaixo do qual uma certa porcentagem dos tempos se encontram)

> Todas as métricas levam em consideração a unidade de segundos para o tempo de resposta da requisição

In [5]:
# Definição dos prompts e endpoint =======================
API_URL = "https://zetta-faunlike-kindheartedly.ngrok-free.dev"
ENDPOINT_TEST = f"{API_URL}/summarize"

PROMPTS = [
    """A energia solar tem se destacado como uma das fontes renováveis mais promissoras para o futuro energético do planeta. Com a crescente preocupação em relação às mudanças climáticas e à necessidade de reduzir a dependência de combustíveis fósseis, muitos países têm investido em tecnologias solares para gerar eletricidade de forma limpa e sustentável. O Brasil, por exemplo, possui um dos maiores potenciais solares do mundo, especialmente nas regiões Nordeste e Centro-Oeste, onde a incidência de radiação solar é alta durante todo o ano. Nos últimos anos, houve um aumento significativo na instalação de painéis solares em residências, comércios e indústrias, impulsionado por incentivos fiscais e pela redução dos custos dos equipamentos. Além disso, a geração distribuída tem permitido que consumidores se tornem também produtores de energia, contribuindo para a descentralização do sistema elétrico nacional. Apesar dos avanços, ainda existem desafios a serem superados, como a necessidade de investimentos em infraestrutura, armazenamento de energia e políticas públicas mais robustas. No entanto, com o avanço da tecnologia e o engajamento da sociedade, a energia solar tende a ocupar um papel cada vez mais relevante na matriz energética brasileira, promovendo desenvolvimento sustentável e redução de impactos ambientais.""",

    """A cidade de Ouro Preto, localizada no estado de Minas Gerais, é um dos maiores símbolos da arquitetura colonial brasileira e da história do país. Fundada no século XVII durante o ciclo do ouro, ela rapidamente se tornou um dos principais centros econômicos da colônia portuguesa, atraindo milhares de pessoas em busca de riqueza. Suas ruas de pedra, ladeiras íngremes e construções preservadas revelam um passado marcado por opulência, religiosidade e resistência. Ouro Preto abriga algumas das mais impressionantes igrejas barrocas do Brasil, como a Igreja de São Francisco de Assis, projetada por Aleijadinho, e decorada com obras de Mestre Ataíde. O município também foi palco de importantes eventos da Inconfidência Mineira, movimento que buscava a independência do Brasil em relação a Portugal. Hoje, a cidade é reconhecida como Patrimônio Mundial pela UNESCO, sendo um destino turístico que atrai visitantes interessados em história, arte e cultura. Além de seu valor histórico, Ouro Preto possui uma vibrante vida universitária, graças à Universidade Federal de Ouro Preto (UFOP), e promove diversos festivais de arte, música e cinema ao longo do ano. Essa combinação de tradição e juventude faz da cidade um espaço único, onde o passado e o presente convivem em harmonia.""",

    """A inteligência artificial (IA) tem se consolidado como uma das tecnologias mais transformadoras do século XXI, impactando profundamente diversos setores da sociedade. Desde a medicina, onde algoritmos auxiliam no diagnóstico precoce de doenças, até a educação, com sistemas personalizados de ensino adaptativo, a IA está redefinindo a forma como interagimos com o mundo. Na indústria, robôs inteligentes otimizam linhas de produção, enquanto no comércio, assistentes virtuais melhoram a experiência do consumidor. Com algoritmos cada vez mais sofisticados e modelos de linguagem avançados, como os LLMs, a IA também está presente em áreas criativas, gerando textos, imagens e até composições musicais. No entanto, essa evolução tecnológica levanta questões éticas importantes. O uso responsável dos dados, a transparência nos processos de decisão automatizada e o combate ao viés algorítmico são temas centrais em debates acadêmicos e políticos. Além disso, há preocupações sobre o impacto da IA no mercado de trabalho, com a automação substituindo funções humanas em ritmo acelerado. Para garantir que os benefícios da inteligência artificial sejam amplamente distribuídos, é essencial que governos, empresas e sociedade civil colaborem na criação de políticas públicas robustas, promovendo inclusão digital, educação tecnológica e regulamentações que assegurem o uso ético e seguro da IA."""
]

In [6]:
# Fazendo requisições e armazenando tempo de resposta ===============
import requests
import time
import csv
# cada prompt é testado e armazenado em um .csv contendo as colunas de: prompt de entrada, resposta de saída, tamanho do prompt e tempo de execução

# Para esse modelo a RESPOSTA DE SAÍDA é indiferente, pois possui parâmetros padrões determinísticos, evitando variações criativas
# cada prompt deve fazer 30 requisições para que seja calculada e analisadas métricas como:
# - média de tempo de prompt => tempo médio por requisição
# - valores máximos e mínimos => valores máximos e mínimos de tempos de requisição para cada prompt
# - desvio padrão => informa a dispersão absoluta dos dados em relação à média
# - coeficiente de variação => informa o quão dispersos estão os dados relativos à média
# - percentis => informa o valor a baixo do qual uma certaporcentagem dos dados se encontra, como p95 (tempo  de resposta que 95% das requisições ficaram abaixo) e p99 (tempo de resposta que 99% das requisições ficaram abaixo)

# Tem problema os prompts resultarem na mesma saída, dado uma mesma entrada? (justificativa do modelo). Seria importante que o modelopossua variações criativas?

CSV_REQ_TIMES = "tempo_requisicoes.csv"
REQS_PER_PROMPT = 30

OPERATION_TIME = 0

# Adicionar try catch aqui para resposta com exceção
with open(CSV_REQ_TIMES, mode="w", newline="", encoding="utf-8") as file:
  writer = csv.writer(file)
  writer.writerow([
      "prompt_id",
      "req_id",
      "prompt_entrada",
      "saída_resposta",
      "tamanho_prompt",
      "tamanho_saida",
      "tempo_s"
  ])

  for pid, prompt in enumerate(PROMPTS, start=1):
    for req_id in range(1, REQS_PER_PROMPT+1):
      try:
        start_req = time.time()
        response = requests.post(ENDPOINT_TEST, json={"text":prompt})
        end_req = time.time()

        # devo armazenar a resposta ??
        text_response = response.json()
        total_time = round(end_req - start_req, 4)

        OPERATION_TIME += total_time

        writer.writerow([
            pid,
            req_id,
            prompt,
            text_response["summary"],
            len(prompt),
            len(text_response["summary"]),
            total_time
        ])
      except Exception as e:
          print(f"Houve um problema com o teste das requisições, prompt/{pid} req/{req_id}:\n {e}")
          break
    print(f"Tempos de resposta do prompt {pid} registrado.")

print("\nDados de tempo de resposta escritos com sucesso.")
print(f"Tempo total da operação: {round(OPERATION_TIME, 4)}s")

Tempos de resposta do prompt 1 registrado.
Tempos de resposta do prompt 2 registrado.
Tempos de resposta do prompt 3 registrado.

Dados de tempo de resposta escritos com sucesso.
Tempo total da operação: 509.2845


In [9]:
# Fazendo análise e registrando dados ==========================
import statistics
import numpy as np
import csv

CSV_RAW = "tempo_requisicoes.csv"
CSV_REQ_TIMES_METRICS = "metricas_tempo_requisicoes.csv"

# dicionário com o array de todos os tempos de requisição
TIMES_REQ_PER_PROMPT_ALL = {}

# armazenar pid, time e prompt_size no dicionário através do csv já existente
with open(CSV_RAW, mode="r", newline="", encoding="utf-8") as file:
  reader = csv.DictReader(file)
  for row in reader:
    pid = int(row["prompt_id"])
    time = float(row["tempo_s"])
    prompt_size = int(row["tamanho_prompt"])

    if pid not in TIMES_REQ_PER_PROMPT_ALL:
      TIMES_REQ_PER_PROMPT_ALL[pid] = {"tempos": [], "tamanho_prompt": prompt_size}
    # Não seria possível armazenar o array de uma vez no dicionário
    TIMES_REQ_PER_PROMPT_ALL[pid]["tempos"].append(time)

# criação do csv as métricas de cada prompt
with open(CSV_REQ_TIMES_METRICS, mode="w", newline="", encoding="utf-8") as  file:
  writer = csv.writer(file)
  writer.writerow([
      "prompt_id",
      "tamanho_prompt",
      "tempo_req_min_s",
      "tempo_req_max_s",
      "tempo_total_s",
      "req_por_prompt",
      "media_tempo_s",
      "desvio_padrao_s",
      "coef_var_%",
      "p95_s",
      "p99_s"
  ])

  for pid, data in TIMES_REQ_PER_PROMPT_ALL.items():
    times = data["tempos"]

    prompt_size = data["tamanho_prompt"]
    min_time = min(times)
    max_time = max(times)
    total_time= round(sum(times), 4)
    req_per_prompt = len(times)
    avg = round(statistics.mean(times), 4)
    std = round(statistics.pstdev(times), 4) # pst x std ???
    coef_var = round(std/avg, 4)
    p95 = round(float(np.percentile(times, 95)), 4)
    p99 = round(float(np.percentile(times, 99)), 4)

    writer.writerow([
        pid,
        prompt_size,
        min_time,
        max_time,
        total_time,
        req_per_prompt,
        avg,
        std,
        coef_var,
        p95,
        p99
    ])

print("Análise e cálculo de métricas finalizado")

Análise e cálculo de métricas finalizado


In [10]:
from google.colab import data_table
import pandas as pd

data_table.enable_dataframe_formatter()
df = pd.read_csv("metricas_tempo_requisicoes.csv")

print("\nMétricas de cada requisição:")
display(df)


Métricas de cada requisição:


,prompt_id,tamanho_prompt,tempo_req_min_s,tempo_req_max_s,tempo_total_s,req_por_prompt,media_tempo_s,desvio_padrao_s,coef_var_%,p95_s,p99_s
0,1,1329,2.3631,8.8799,104.4081,30,3.4803,1.3299,0.3821,5.9832,8.3577
1,2,1279,3.2984,60.9351,225.5355,30,7.5179,11.0010,1.4633,21.5667,51.4343
2,3,1390,2.3712,32.9322,179.3409,30,5.9780,6.7923,1.1362,20.2238,30.3483
